In [ ]:
from datascience import *
%matplotlib inline
path_data = '../../../assets/data/'
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=np.VisibleDeprecationWarning)

## Lecture 12 ##

## Intro to Join ##

In [ ]:
drinks = Table(['Drink', 'Cafe', 'Price'])
drinks = drinks.with_rows([
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', 'Strada',  1.75],
    ['Latte',    'Strada',  3.25],
    ['Espresso', "FSM",   2]
])
drinks

In [ ]:
discounts = Table().with_columns(
    'Coupon % off', make_array(10, 25, 5),
    'Location', make_array('Asha', 'Strada', 'Asha')
)
discounts

In [ ]:
# tbl1.join(label_from_tbl1, tbl2, label_from_tbl2)

combined = drinks.join('Cafe', discounts, 'Location')
combined

In [ ]:
discounted_frac = 1 - combined.column('Coupon % off') / 100
combined.with_column(
    'Discounted Price', 
    combined.column('Price') * discounted_frac
)

## The Result is Sorted ##

In [ ]:
drinks2 = Table(['Drink', 'Cafe', 'Price'])
drinks2 = drinks2.with_rows([
    ['Latte',    'Strada',  3.25],
    ['Espresso', 'Strada',  1.75],
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', "FSM",   2]
])
drinks2

In [ ]:
discounts

In [ ]:
drinks2.join('Cafe', discounts, 'Location')

## Notes on Column Names ##

In [ ]:
drinks

In [ ]:
drinks_new = Table().with_columns(
    'Drink', make_array('Tea', 'Coffee', 'Coffee'),
    'Cafe', make_array('Asha', 'FSM', 'Strada')
)
drinks_new

In [ ]:
# 'Drink' column of drinks_new is relabeled Drink_2

drinks.join('Cafe', drinks_new, 'Cafe')

In [ ]:
# If you don't specify a column label in the second table,
# join looks for a column with the same label 
# as specified in the first argument

drinks.join('Cafe', drinks_new)

## Another Join Example ##

In [ ]:
full = Table.read_table('nc-est2019-agesex-res.csv')
census = full.select('SEX', 'AGE', 'POPESTIMATE2019')
census

In [ ]:
sex_codes = Table().with_columns(
    'SEX CODE', make_array(0, 1, 2),
    'CODE DEFINITION', make_array('All', 'Selected Male', 'Selected Female')
)
sex_codes

In [ ]:
sex_codes.join('SEX CODE', census, 'SEX').show()

## Bike Sharing ##

In [ ]:
trip = Table.read_table('trip.csv')
trip

## Distribution of Durations ##

In [ ]:
trip.hist('Duration')

In [ ]:
trip.sort('Duration', descending=True)

In [ ]:
commute = trip.where('Duration', are.below(1800))
commute.hist('Duration')

In [ ]:
commute.hist('Duration', bins=np.arange(0, 1800, 250), unit='Second')

In [ ]:
# Approx percent of people who have 
# a ride duration between 250 and 500 seconds
# "between" = [250, 500) 

(500-250) * 0.15 

In [ ]:
commute.where('Duration', are.between(250, 500)).num_rows

In [ ]:
commute.num_rows

In [ ]:
129079 / 338343

In [ ]:
commute.hist('Duration', bins=np.arange(0, 1800, 250), unit='Second')

In [ ]:
commute.hist('Duration', bins=60, unit='Second')

## Start and End Stations ##

In [ ]:
# Most common start station

starts = commute.group('Start Station').sort('count', descending=True)
starts

In [ ]:
# Numbers of trips between stations

commute.pivot('Start Station', 'End Station')

In [ ]:
# Average durations of trips between stations

commute.pivot('Start Station', 'End Station', values='Duration', collect=np.average)

## Fastest Trips between Stations ##

In [ ]:
duration = trip.select('Start Station', 'End Station', 'Duration')
duration

In [ ]:
# Durations of fastest trips between stations,
# using group

shortest = duration.group(['Start Station', 'End Station'], min)
shortest

In [ ]:
# Duration of fastest trip between stations,
# using pivot

duration.pivot('Start Station', 'End Station', values='Duration', collect=min)

In [ ]:
# Shortest trips from Civic Center
from_cc = shortest.where(
    'Start Station', are.containing('Civic Center BART')).sort(
    'Minimum Duration')
from_cc

## Maps ##

In [ ]:
# Geographical data on the stations
stations = Table.read_table('station.csv').drop(4, 6)
stations

In [ ]:
sf_stations = stations.where('landmark', are.equal_to('San Francisco'))
sf_stations_map_data = sf_stations.select(
    'lat', 'long', 'name').relabeled(
    'name', 'labels'
)
sf_stations_map_data.show(3)

In [ ]:
Marker.map_table(sf_stations_map_data)

In [ ]:
Circle.map_table(sf_stations_map_data)

In [ ]:
starts = commute.group('Start Station').sort('count', descending=True)
starts.show(3)

In [ ]:
stations.show(3)

In [ ]:
station_starts = stations.join('name', starts, 'Start Station')
station_starts.show(3)

In [ ]:
sf_starts = station_starts.relabeled(
    'name', 'labels').where('landmark', 'San Francisco')

In [ ]:
sf_starts_map_data = sf_starts.select('lat', 'long', 'labels').with_columns(
    'colors', 'blue',
    'areas', sf_starts.column('count')
)
sf_starts_map_data.show(3)

In [ ]:
Circle.map_table(sf_starts_map_data)